In [1]:
#importando biblioteca pandas
import pandas as pd

###Visualização de Dados
Nesta parte estamos analisando diferentes arquivos em Excel e iremos montar um df(dataframe) único com os dados das 5 planilhas.

In [2]:
df1 = pd.read_excel('/content/drive/MyDrive/Analise de dados com Pandas/Aracaju.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/Analise de dados com Pandas/Fortaleza.xlsx')
df3 = pd.read_excel('/content/drive/MyDrive/Analise de dados com Pandas/Natal.xlsx')
df4 = pd.read_excel('/content/drive/MyDrive/Analise de dados com Pandas/Recife.xlsx')
df5 = pd.read_excel('/content/drive/MyDrive/Analise de dados com Pandas/Salvador.xlsx')

Aqui estamos concatenando as planilhas

In [3]:
df = pd.concat([df1,df2,df3,df4,df5])
df.head()

,Cidade,Data,Vendas,LojaID,Qtde
0,Aracaju,2018-01-01,142.00,1520,1
1,Aracaju,2018-01-01,14.21,1522,6
2,Aracaju,2018-01-01,71.55,1520,1
3,Aracaju,2018-01-01,3.01,1521,7
4,Aracaju,2018-01-01,24.51,1522,8


Aqui estamos verificando o tipo de dados de cada coluna

In [5]:
df.dtypes

Cidade            object
Data      datetime64[ns]
Vendas           float64
LojaID             int64
Qtde               int64
dtype: object

Agora que oservamos o tipo de dados e estão todos em formato adequado iremos começar algumas análise a fim de treinar as ferramentas fornecidas pelo pandas.

1) Verificar se há valores faltantes na base de dados.

In [6]:
df.isnull().sum()

Cidade    0
Data      0
Vendas    0
LojaID    0
Qtde      0
dtype: int64

Esta base já havia sido tratada, mas colocarei aqui algumas opções de tratamento. São elas:

1)Substituindo os valores faltantes pela média.

Ex.: df["Vendas"].fillna(df["Vendas"].mean(), inplace=True)

obs.: Nem sempre isso faz sentido. Precisa observar o tipo de negócio, o range de valores (nesse caso) e a quantidade de valores faltantes. As vezes o melhor pode ser usar a mediana, por exemplo.

2)Substituindo os valores faltantes por zero.

ex.: df["Vendas"].fillna(0, inplace=True)

3)Apagando as linhas com valores faltantes. Nesse caso iremos toda linha que apresentar um valor faltante.

Ex.: df.dropna(inplace=True)

Obs.: Este talvez faça mais sentido na maioria dos casos pois não irá interferir na análise estatística.

4)Apagando as linhas com valores faltantes com base apenas em 1 coluna.

Ex.: df.dropna(subset=["Vendas"], inplace=True)

5)Removendo linhas que estejam com valores faltantes em todas as colunas.

Ex.: df.dropna(how="all", inplace=True)

Criando colunas novas.

In [7]:
df['Receita'] = df['Vendas'] * df['Qtde']

In [9]:
df.sample(10)

,Cidade,Data,Vendas,LojaID,Qtde,Receita
74,Recife,2019-01-01,38.79,983,6,232.74
222,Natal,2019-01-02,43.96,1037,1,43.96
159,Natal,2019-01-02,172.52,1035,1,172.52
200,Natal,2019-01-02,41.06,1035,1,41.06
101,Fortaleza,2019-01-01,11.67,1005,4,46.68
217,Salvador,2019-01-02,37.65,1036,2,75.30
239,Natal,2019-01-03,161.41,1037,3,484.23
101,Aracaju,2018-01-01,16.62,1520,9,149.58
156,Salvador,2019-01-02,36.14,1037,3,108.42
107,Recife,2019-01-01,15.40,980,7,107.80


Vamos analisar:

1) Receita por ano

2) Receita por filial

3) Receita por filial/ano

4) Se há um mês ou dia melhor para o faturamento da empresa e de cada unidade.

In [11]:
# receitas por ano
df.groupby(df['Data'].dt.year)['Receita'].sum()

Data
2018    118176.53
2019    228246.45
Name: Receita, dtype: float64

In [12]:
# receita por filial
df.groupby(df['Cidade'])['Receita'].sum()

Cidade
Aracaju       48748.25
Fortaleza     37913.97
Natal        167227.52
Recife        51936.51
Salvador      40596.73
Name: Receita, dtype: float64

In [14]:
# receita por filial por ano
df.groupby([df['Cidade'],df['Data'].dt.year])['Receita'].sum()

Cidade     Data
Aracaju    2018    46276.29
           2019     2471.96
Fortaleza  2019    37913.97
Natal      2018    71750.00
           2019    95477.52
Recife     2019    51936.51
Salvador   2018      150.24
           2019    40446.49
Name: Receita, dtype: float64

In [16]:
#Resetando o index
lucro_ano = df.groupby([df['Cidade'],df['Data'].dt.year])['Receita'].sum().reset_index()
lucro_ano


,Cidade,Data,Receita
0,Aracaju,2018,46276.29
1,Aracaju,2019,2471.96
2,Fortaleza,2019,37913.97
3,Natal,2018,71750.00
4,Natal,2019,95477.52
5,Recife,2019,51936.51
6,Salvador,2018,150.24
7,Salvador,2019,40446.49


In [18]:
#analisando se houve um mês melhor para o faturamento
mes_venda = df.groupby(df['Data'].dt.month)['Receita'].sum()
mes_venda

Data
1     188456.50
2      37244.47
3      67794.01
4      13586.00
5       8965.00
6       7097.00
7       4462.00
8       1212.00
9       5984.00
10      9680.00
11      1126.00
12       816.00
Name: Receita, dtype: float64

In [26]:
#analisando se houve um dia melhor para o faturamento
dia_venda = df.groupby(df['Data'].dt.day)['Receita'].sum()
dia_venda.sort_values(ascending=False)

Data
1     132810.21
2      84283.85
3      13464.02
20     10402.00
27     10304.00
10     10075.80
8       8522.00
14      8328.65
22      6242.00
13      6199.16
24      6086.00
11      5752.60
9       5737.00
7       5625.00
18      5476.00
21      5299.00
25      4438.00
12      4185.69
30      2760.00
28      2228.00
5       2130.00
23      1608.00
26      1506.00
16      1170.00
15       738.00
31       564.00
17       488.00
Name: Receita, dtype: float64

Vamos agora incluir mais algumas colunas para exercitar e depois gerar gráficos.